In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib as mpl
import nltk.classify.util
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.classify import NaiveBayesClassifier
import numpy as np
import re
import string
import nltk
%matplotlib inline

In [3]:
df = pd.read_csv("1429_1.csv")
df.head()


/home/youssef/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5620406/review...,I've had my Fire HD 8 two weeks now and I love...,Good!!!,NaN,NaN,Shacks
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,NaN,NaN,explore42


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34660 entries, 0 to 34659
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    34660 non-null  object 
 1   name                  27900 non-null  object 
 2   asins                 34658 non-null  object 
 3   brand                 34660 non-null  object 
 4   categories            34660 non-null  object 
 5   keys                  34660 non-null  object 
 6   manufacturer          34660 non-null  object 
 7   reviews.date          34621 non-null  object 
 8   reviews.dateAdded     24039 non-null  object 
 9   reviews.dateSeen      34660 non-null  object 
 10  reviews.didPurchase   1 non-null      object 
 11  reviews.doRecommend   34066 non-null  object 
 12  reviews.id            1 non-null      float64
 13  reviews.numHelpful    34131 non-null  float64
 14  reviews.rating        34627 non-null  float64
 15  reviews.sourceURLs 

In [5]:
df['reviews.text'] = df['reviews.text'].astype('str')
df['reviews.text'] = df['reviews.text'].str.replace('/',' ')
df['reviews.text'] = df['reviews.text'].apply(lambda x: str(x))


In [6]:
df["reviews.text"] = df["reviews.text"].replace(["\:\)", "\:\-\)", "\:\-\}", "\;\-\}", "\:\-\>", "\;\-\)"], ["Happy","Happy","Happy","Happy","Happy","Happy"], regex=True)
df["reviews.text"] = df["reviews.text"].replace(["\:\-\(", "\:\(", "\:\-\|", "\;\-\(", "\;\-\<", "\|\-\{"], ["Sad", "Sad", "Sad", "Sad", "Sad", "Sad",], regex=True)
df["reviews.text"] = df["reviews.text"].replace(["\:\D", "\:\'\-\)", "\:\`\-\("], ["laugh", "tear of joy", "tear of sadness"], regex=True)


In [7]:
import string
exclude = set(string.punctuation)
final = exclude

for i in final:
  df['reviews.text'] = df['reviews.text'].str.replace(i, ' ', regex=True)

In [8]:
permanent = df[['reviews.rating', 'reviews.text', 'reviews.title', 'reviews.username']]

In [9]:
print(permanent.isnull().sum())
permanent.head()

reviews.rating      33
reviews.text         0
reviews.title        5
reviews.username     2
dtype: int64


,reviews.rating,reviews.text,reviews.title,reviews.username
0,5.0,This product so far has not disappointed My c...,Kindle,Adapter
1,5.0,great for beginner or experienced person Boug...,very fast,truman
2,5.0,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,DaveZ
3,4.0,I ve had my Fire HD 8 two weeks now and I love...,Good!!!,Shacks
4,5.0,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,explore42


In [10]:
senti= permanent[permanent["reviews.rating"].notnull()]

In [11]:
senti.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 34627 entries, 0 to 34659
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   reviews.rating    34627 non-null  float64
 1   reviews.text      34627 non-null  object 
 2   reviews.title     34622 non-null  object 
 3   reviews.username  34625 non-null  object 
dtypes: float64(1), object(3)
memory usage: 1.3+ MB


In [13]:
from stop_words import get_stop_words
from nltk.corpus import stopwords

stop_words = list(get_stop_words('en'))         #About 900 stopwords
nltk_words = list(stopwords.words('english')) #About 179 stopwords
stop_words.extend(nltk_words)


In [14]:
from nltk.corpus import stopwords 
stop = set(stopwords.words('english')) 
from nltk.corpus import stopwords 
def remove_stopword(word): 
    return word not in words 
 
senti['reviewsStopWords'] = senti['reviews.text'].str.lower().str.split()

<ipython-input-14-1e53e9df03a6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  senti['reviewsStopWords'] = senti['reviews.text'].str.lower().str.split()


In [15]:
import nltk
nltk.download('punkt')
senti['reviews.text'] = df['reviews.text'].astype(str).str.replace('/',' ')

[nltk_data] Downloading package punkt to /home/youssef/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-15-0d97385252db>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  senti['reviews.text'] = df['reviews.text'].astype(str).str.replace('/',' ')


In [16]:
senti["reviews.text"] = senti["reviews.text"].apply(nltk.word_tokenize)


<ipython-input-16-188b96a1dc4a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  senti["reviews.text"] = senti["reviews.text"].apply(nltk.word_tokenize)


In [17]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [18]:
senti['reviews.text'] = senti['reviews.text'].apply(lambda x: [stemmer.stem(y) for y in x])

<ipython-input-18-2b91b018ee93>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  senti['reviews.text'] = senti['reviews.text'].apply(lambda x: [stemmer.stem(y) for y in x])


In [19]:
senti["senti"] = senti["reviews.rating"]>=4
senti["senti"] = senti["senti"].replace([True , False] , [1 , -1])

<ipython-input-19-ecd492c4e3a9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  senti["senti"] = senti["reviews.rating"]>=4
<ipython-input-19-ecd492c4e3a9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  senti["senti"] = senti["senti"].replace([True , False] , [1 , -1])


In [20]:
data_set = senti[["reviews.text", "senti"]]
data_set.columns = ["reviews", "score"]
data_set.head()

,reviews,score
0,"[this, product, so, far, has, not, disappoint,...",1
1,"[great, for, beginn, or, experienc, person, bo...",1
2,"[inexpens, tablet, for, him, to, use, and, lea...",1
3,"[i, ve, had, my, fire, hd, 8, two, week, now, ...",1
4,"[i, bought, this, for, my, grand, daughter, wh...",1


In [21]:
data_set["reviews"] = data_set["reviews"].apply(' '.join)

<ipython-input-21-571c72e26155>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set["reviews"] = data_set["reviews"].apply(' '.join)


In [22]:
train = data_set
train.columns = ["reviews", "score"]
train.head()

,reviews,score
0,this product so far has not disappoint my chil...,1
1,great for beginn or experienc person bought as...,1
2,inexpens tablet for him to use and learn on st...,1
3,i ve had my fire hd 8 two week now and i love ...,1
4,i bought this for my grand daughter when she c...,1


In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
#tokenizer to remove unwanted elements from out data like symbols and numbers
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts= cv.fit_transform(train['reviews'])

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text_counts, train['score'], test_size=0.3, random_state=1)

In [26]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, y_train)
predicted= clf.predict(X_test)

In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

          -1       0.48      0.29      0.36       685
           1       0.95      0.98      0.96      9704

    accuracy                           0.93     10389
   macro avg       0.71      0.63      0.66     10389
weighted avg       0.92      0.93      0.92     10389

